In [1]:
# #List of tickers in industry we are currently processing 
# current_industry_list = ['MSFT', 'TSLA']


# #==========================================================================================
# #==========================================================================================
# # Step 1: extract the returns_df with only these ones 
# subset_returns_df = returns_df[current_industry_list]

# #Returns done!



# #==========================================================================================
# #==========================================================================================
# # Step 2: cum return df 
# # Initialize a list to store cum Series
# cum_returns_list = []


# t.start()

# for ticker in subset_returns_df.columns:
#     # Calculate cumulative returns
#     cum_returns = pd.Series(index=subset_returns_df.index, dtype='float64')
#     started = False  # Flag to indicate when the calculation starts
#     for i in range(len(subset_returns_df)):
#         if pd.notna(subset_returns_df[ticker].iloc[i]):
#             if not started:
#                 # Initialize with the first valid return
#                 cum_returns.iloc[i] = 1 + subset_returns_df[ticker].iloc[i]
#                 started = True
#             else:
#                 # Calculate cumulative return
#                 cum_returns.iloc[i] = cum_returns.iloc[i-1] * (1 + subset_returns_df[ticker].iloc[i])
#         else:
#             # Carry forward the last valid cumulative return
#             if started:
#                 cum_returns.iloc[i] = cum_returns.iloc[i-1]
    
#     # Append the cumulative returns Series to the list, with ticker as the name
#     cum_returns.name = ticker
#     cum_returns_list.append(cum_returns)

# # Concatenate all cumulative returns Series into a single DataFrame
# subset_cum_stock_returns_df = pd.concat(cum_returns_list, axis=1)

# t.end()


# #==========================================================================================
# #==========================================================================================
# # Step 3: extract relevant market_cap_df data 
# subset_market_cap_df = market_cap_df[current_industry_list]
# #Add total market cap
# subset_market_cap_df['Total Market Cap'] = subset_market_cap_df.sum(axis=1)


# #==========================================================================================
# #==========================================================================================
# # Step 4: Get weights with new subset 
# t.start()

# # Initialize weights_df with the same data as market_cap_df
# subset_weights_df = subset_market_cap_df.copy()


# #Compute weights directly using broadcasting (divides eveything by specified Total_Market_Cap column)
# subset_weights_df = subset_weights_df.divide(subset_weights_df['Total Market Cap'], axis=0)

# subset_weights_df.rename(columns={'Total Market Cap': 'Total Weight'}, inplace=True)


# t.end()


# #==========================================================================================
# #==========================================================================================
# # Step 5: Build indexes 

# #First, cut last column to make dimensions OK
# new_weights_df = subset_weights_df.drop(columns='Total Weight')

# #Fill NaN with 0 for math 
# new_returns_df = subset_returns_df.fillna(0)
# new_weights_df = new_weights_df.fillna(0)


# #Transpose the weights
# transposed_weights_df = new_weights_df.T

# #Match indices
# transposed_weights_df.index = subset_returns_df.columns


# #Compute matrix multiplication 
# result_df = subset_returns_df.dot(transposed_weights_df)

# if result_df.shape[0] == result_df.shape[1]:
#     # Extract diagonal elements as a NumPy array
#     diagonal_elements = np.diag(result_df)
    
#     # Convert the NumPy array to a DataFrame with a single column
#     subset_index_df = pd.DataFrame(diagonal_elements, columns=['Diagonal'])
    

# #Have index_df be with good index
# # Initialize cumulative_index_df with NaNs and the same index as returns_df (which is indexed by dates)
# subset_index_df = pd.DataFrame(index=subset_returns_df.index, columns=['Diagonal'])

# # Compute matrix multiplication 
# result_df = subset_returns_df.dot(transposed_weights_df)

# # Extract diagonal elements and assign to the index_df
# if result_df.shape[0] == result_df.shape[1]:
#     diagonal_elements = np.diag(result_df)
#     subset_index_df['Diagonal'] = diagonal_elements 
    
    
# #==========================================================================================
# #==========================================================================================
# # Step 6: Build cum indexes 

# # Initialize cumulative_index_df with NaNs and the same index as cum_stock_returns_df (which is indexed by dates)
# subset_cum_index_df = pd.DataFrame(index=subset_cum_stock_returns_df.index, columns=['Cumulative'])

# # Set the first row of cumulative_index_df
# subset_cum_index_df.iloc[0] = 1 + subset_index_df['Diagonal'].iloc[0]

# # Populate the rest of cumulative_index_df
# for i in range(1, len(subset_cum_index_df)):
#     subset_cum_index_df.iloc[i] = subset_cum_index_df.iloc[i - 1] * (1 + subset_index_df['Diagonal'].iloc[i])

# # Print out the cumulative index DataFrame
# print('Cumulative Index:')
# subset_cum_index_df

In [3]:
import pandas as pd
import numpy as np

import sys
import os 

#Completely optional, this helps time different parts of the code 
# sys.path.append(r'C:\Users\holsg\Documents\Python\Script Uni\Homework\Level 2 - Copy\Section 2.1\2.1.1 to 2.1.5')

from timer_class import Timer

t = Timer()

def process_industry_subset(subset_list, returns_df, market_cap_df):
    # Step 1: Extract the returns_df with only these tickers
    subset_returns_df = returns_df[subset_list]
    
    print('Step 1 done: Returns dataframe.')
    
    # Step 2: Calculate cumulative returns
    cum_returns_list = []
    
    t.start()  # Assuming this is for timing, replace with actual timing code if necessary
    
    for ticker in subset_returns_df.columns:
        cum_returns = pd.Series(index=subset_returns_df.index, dtype='float64')
        started = False
        for i in range(len(subset_returns_df)):
            if pd.notna(subset_returns_df[ticker].iloc[i]):
                if not started:
                    cum_returns.iloc[i] = 1 + subset_returns_df[ticker].iloc[i]
                    started = True
                else:
                    cum_returns.iloc[i] = cum_returns.iloc[i-1] * (1 + subset_returns_df[ticker].iloc[i])
            else:
                if started:
                    cum_returns.iloc[i] = cum_returns.iloc[i-1]
        
        cum_returns.name = ticker
        cum_returns_list.append(cum_returns)
    
    subset_cum_stock_returns_df = pd.concat(cum_returns_list, axis=1)
    
    print('Step 2 done: Cumulative Returns dataframe.')
    
    t.end()  # Assuming this is for timing, replace with actual timing code if necessary
    
    # Step 3: Extract relevant market_cap_df data
    subset_market_cap_df = market_cap_df[subset_list]
    subset_market_cap_df['Total Market Cap'] = subset_market_cap_df.sum(axis=1)
    
    print('Step 3 done: Market cap dataframe.')
    
    # Step 4: Get weights with the new subset
    t.start()  # Assuming this is for timing, replace with actual timing code if necessary
    
    subset_weights_df = subset_market_cap_df.copy()
    subset_weights_df = subset_weights_df.divide(subset_weights_df['Total Market Cap'], axis=0)
    subset_weights_df.rename(columns={'Total Market Cap': 'Total Weight'}, inplace=True)
    
    print('Step 4 done: Weights dataframe.')
    
    t.end()  # Assuming this is for timing, replace with actual timing code if necessary
    
    # Step 5: Build indexes
    new_weights_df = subset_weights_df.drop(columns='Total Weight')
    new_returns_df = subset_returns_df.fillna(0)
    new_weights_df = new_weights_df.fillna(0)
    
    transposed_weights_df = new_weights_df.T
    transposed_weights_df.index = subset_returns_df.columns
    
    result_df = subset_returns_df.dot(transposed_weights_df)
    
    if result_df.shape[0] == result_df.shape[1]:
        diagonal_elements = np.diag(result_df)
        subset_index_df = pd.DataFrame(diagonal_elements, columns=['Diagonal'])
    else:
        subset_index_df = pd.DataFrame(index=subset_returns_df.index, columns=['Diagonal'])
    
    # Compute matrix multiplication again
    result_df = subset_returns_df.dot(transposed_weights_df)
    
    if result_df.shape[0] == result_df.shape[1]:
        diagonal_elements = np.diag(result_df)
        subset_index_df['Diagonal'] = diagonal_elements
    
    print('Step 5 done: Index dataframe.')
    
    # Step 6: Build cumulative indexes
    subset_cum_index_df = pd.DataFrame(index=subset_cum_stock_returns_df.index, columns=['Cumulative'])
    subset_cum_index_df.iloc[0] = 1 + subset_index_df['Diagonal'].iloc[0]
    
    for i in range(1, len(subset_cum_index_df)):
        subset_cum_index_df.iloc[i] = subset_cum_index_df.iloc[i - 1] * (1 + subset_index_df['Diagonal'].iloc[i])
        
    print('Step 6 done: Cumulative Indeex dataframe. ')
    print('\n----- Returning values -----')
    
    return (subset_returns_df, subset_cum_stock_returns_df, subset_market_cap_df, 
            subset_weights_df, subset_index_df, subset_cum_index_df)


In [4]:
### ONLY FIRST 4  DATAFRAMES

import pandas as pd
import numpy as np

import sys
import os 

#Completely optional, this helps time different parts of the code 
# sys.path.append(r'C:\Users\holsg\Documents\Python\Script Uni\Homework\Level 2 - Copy\Section 2.1\2.1.1 to 2.1.5')

from timer_class import Timer

t = Timer()

def process_industry_subset(subset_list, returns_df, market_cap_df):
    # Step 1: Extract the returns_df with only these tickers
    subset_returns_df = returns_df[subset_list]
    
    print('Step 1 done: Returns dataframe.')
    
    # Step 2: Calculate cumulative returns
    cum_returns_list = []
    
    t.start()  # Assuming this is for timing, replace with actual timing code if necessary
    
    for ticker in subset_returns_df.columns:
        cum_returns = pd.Series(index=subset_returns_df.index, dtype='float64')
        started = False
        for i in range(len(subset_returns_df)):
            if pd.notna(subset_returns_df[ticker].iloc[i]):
                if not started:
                    cum_returns.iloc[i] = 1 + subset_returns_df[ticker].iloc[i]
                    started = True
                else:
                    cum_returns.iloc[i] = cum_returns.iloc[i-1] * (1 + subset_returns_df[ticker].iloc[i])
            else:
                if started:
                    cum_returns.iloc[i] = cum_returns.iloc[i-1]
        
        cum_returns.name = ticker
        cum_returns_list.append(cum_returns)
    
    subset_cum_stock_returns_df = pd.concat(cum_returns_list, axis=1)
    
    print('Step 2 done: Cumulative Returns dataframe.')
    
    t.end()  # Assuming this is for timing, replace with actual timing code if necessary
    
    # Step 3: Extract relevant market_cap_df data
    subset_market_cap_df = market_cap_df[subset_list]
    subset_market_cap_df['Total Market Cap'] = subset_market_cap_df.sum(axis=1)
    
    print('Step 3 done: Market cap dataframe.')
    
    # Step 4: Get weights with the new subset
    t.start()  # Assuming this is for timing, replace with actual timing code if necessary
    
    subset_weights_df = subset_market_cap_df.copy()
    subset_weights_df = subset_weights_df.divide(subset_weights_df['Total Market Cap'], axis=0)
    subset_weights_df.rename(columns={'Total Market Cap': 'Total Weight'}, inplace=True)
    
    print('Step 4 done: Weights dataframe.')
    print('\n----- Returning values -----')
    
    
    t.end()  # Assuming this is for timing, replace with actual timing code if necessary
    
    return (subset_returns_df, subset_cum_stock_returns_df, subset_market_cap_df, 
            subset_weights_df)
    



In [5]:
#NESXT 2:



def compute_indices(subset_returns_df, subset_weights_df, subset_cum_stock_returns_df):
    # Drop the 'Total Weight' column and fill NaNs with 0
    new_weights_df = subset_weights_df.drop(columns='Total Weight').fillna(0)
    subset_returns_df = subset_returns_df.fillna(0)

    # Transpose the weights DataFrame
    transposed_weights_df = new_weights_df.T
    transposed_weights_df.index = subset_returns_df.columns

    # Compute matrix multiplication
    result_df = subset_returns_df.dot(transposed_weights_df)

    # Initialize index_df with NaNs
    index_df = pd.DataFrame(index=subset_returns_df.index, columns=['Diagonal'])

    if result_df.shape[0] == result_df.shape[1]:
        # Extract diagonal elements and assign to index_df
        diagonal_elements = np.diag(result_df)
        index_df['Diagonal'] = diagonal_elements

    # Initialize cumulative_index_df
    cum_index_df = pd.DataFrame(index=subset_cum_stock_returns_df.index, columns=['Cumulative'])

    # Set the first row of cumulative_index_df
    cum_index_df.iloc[0] = 1 + index_df['Diagonal'].iloc[0]

    # Populate the rest of cumulative_index_df
    for i in range(1, len(cum_index_df)):
        cum_index_df.iloc[i] = cum_index_df.iloc[i - 1] * (1 + index_df['Diagonal'].iloc[i])

    # Return both DataFrames
    return index_df, cum_index_df

In [ ]:
####NEXT 2 DATAFRAMES 

#Optimize run speed maybe by not computing every value, just the relevant vector 



#Now build the index column df


#This is actually easily computable by treating the dataframes as a matrix multiplication
#Dates are index, not a column, so they do not interfere 
#Also, NaN = boom. Just replace with a 0, and that contributes nothing to index so its as if we ignore it nayways 

#Multiply return df by transpose of weights. We collect the diagonal as our column of indices for each period. 



#First, cut last column to make dimensions OK
new_weights_df = weights_df.drop(columns='Total Weight')


#Fill NaN with 0
returns_df = returns_df.fillna(0)
new_weights_df = new_weights_df.fillna(0)


#Transpose the weights
transposed_weights_df = new_weights_df.T


#Match indices
transposed_weights_df.index = returns_df.columns


#Compute matrix multiplication 
result_df = returns_df.dot(transposed_weights_df)


if result_df.shape[0] == result_df.shape[1]:
    # Extract diagonal elements as a NumPy array
    diagonal_elements = np.diag(result_df)
    
    # Convert the NumPy array to a DataFrame with a single column
    index_df = pd.DataFrame(diagonal_elements, columns=['Diagonal'])
    
    
#Instant 
print('Index:')
index_df

#Have index_df be with good index

# Initialize cumulative_index_df with NaNs and the same index as returns_df (which is indexed by dates)
index_df = pd.DataFrame(index=returns_df.index, columns=['Diagonal'])

# Compute matrix multiplication 
result_df = returns_df.dot(transposed_weights_df)

# Extract diagonal elements and assign to the index_df
if result_df.shape[0] == result_df.shape[1]:
    diagonal_elements = np.diag(result_df)
    index_df['Diagonal'] = diagonal_elements

# Print the updated index_df to verify
print('Index:')
index_df


#Now create cum-index

# Initialize cumulative_index_df with NaNs and the same index as cum_stock_returns_df (which is indexed by dates)
cum_index_df = pd.DataFrame(index=cum_stock_returns_df.index, columns=['Cumulative'])

# Set the first row of cumulative_index_df
cum_index_df.iloc[0] = 1 + index_df['Diagonal'].iloc[0]

# Populate the rest of cumulative_index_df
for i in range(1, len(cum_index_df)):
    cum_index_df.iloc[i] = cum_index_df.iloc[i - 1] * (1 + index_df['Diagonal'].iloc[i])

# Print out the cumulative index DataFrame
print('Cumulative Index:')
cum_index_df